# Run a basic Hermes Hartree-Fock energy calculation on ligand conformers from Auto3d

In this notebook, we'll perform a hermes energy calculation on a ligand derived from a smiles string

# 0) Complete example
See the [sample notebook](/Quickstarts/auto3d_conformer_tautomer_generation-sample.ipynb) for a complete demonstration.

# 1) Setup

## 1.0) Imports

In [ ]:
import json
import os
import sys
import tarfile

from pdbtools import *
import requests
from datetime import datetime
from pathlib import Path
import py3Dmol

import rush

## 1.1) Configuration

In [ ]:
EXPERIMENT = "tengu-py-auto3d_hermes"
LIGAND_SMILES = "CC(=O)OC1=CC=CC=C1C(=O)O "
LIGAND = "ASPRIN"
TAGS = ["qdx", EXPERIMENT, LIGAND]

In [ ]:
# |hide
WORK_DIR = Path.home() / "qdx" / EXPERIMENT

if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)

os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(WORK_DIR / ".rush", exist_ok=True)

os.chdir(WORK_DIR)
YOUR_TOKEN = os.getenv("RUSH_TOKEN")

2024-05-07 17:26:37,603 - rush - WARNING - Module auto3d has a different version on the server: github:talo/tengu-auto3d/19d66c0f29a1c204b0149af0000983874675a272#auto3d_tengu.
                                Use `.update_modules()` to update the lock file
2024-05-07 17:26:37,605 - rush - WARNING - Module gnina_pdb has a different version on the server: github:talo/tengu-gnina/226c2d5b94f4e4773c41a13613d588fad41c0eab#gnina_tengu_pdb.
                                Use `.update_modules()` to update the lock file
2024-05-07 17:26:37,606 - rush - WARNING - Module p2rank has a different version on the server: github:talo/tengu-p2rank/b8b810bc2a40fe507d65a4280922ac576a223519#p2rank_tengu.
                                Use `.update_modules()` to update the lock file
2024-05-07 17:26:37,607 - rush - WARNING - Module p2rank_pdb has a different version on the server: github:talo/tengu-p2rank/b8b810bc2a40fe507d65a4280922ac576a223519#p2rank_pdb_tengu.
                                Use `.updat

## 1.2) Build your client

In [ ]:
# Get our client, for calling modules and using the rush API
client = rush.build_blocking_provider_with_functions(batch_tags=TAGS)

# 2) Preparation
We want to convert our raw files into prepared qdxfs, with correct charges and missing residues filled

In [ ]:
help(client.auto3d)

Help on function auto3d in module rush.provider:

auto3d(*args: *tuple[RushObject[bytes], str, Record], target: 'Target | None' = None, resources: 'Resources | None' = None, tags: 'list[str] | None' = None, restore: 'bool | None' = None) -> tuple[RushObject[bytes], RushObject[list[Record]]]
    Generate 3D conformers from SMILES strings and other inputs

    Module version:
    `github:talo/tengu-auto3d/19d66c0f29a1c204b0149af0000983874675a272#auto3d_tengu`

    QDX Type Description:

        molecule_file: Object[@$Bytes];
        molecule_file_type: string;
        options: Auto3dOptions {
            optimizing_engine: Auto3dOptimizingEngines[ANI2x | ANI2xt | AIMNET]?,
            enumerate_isomer: bool?,
            batchsize_atoms: u32?,
            max_confs: u32?,
            use_gpu: bool?,
            enumerate_tautomer: bool?,
            verbose: bool?,
            memory: u32?,
            patience: u32?,
            capacity: u32?,
            gpu_idx: [u32]?,
            

In [ ]:
ligand_path = client.workspace / "aspirin.smiles"
with open(ligand_path, "w") as ligand_file:
    print(f"{LIGAND_SMILES} {LIGAND_SMILES}", file=ligand_file)

(conformer_sdf, conformer_qdxf) = client.auto3d(
    ligand_path,
    "smi",
    {"k": 5, "use_gpu": True},
    resources={"gpus": 1, "storage": "5", "storage_units": "GB"},
)

print(f"{datetime.now().time()} | Running ligand preparation!")

17:26:40.429981 | Running ligand preparation!


In [ ]:
conformer_qdxf.source

UUID('7fbbd6f9-2b3c-4a45-9e07-51f557eb801b')

In [ ]:
conformer_sdf.value

In [ ]:
try:
    conformer_sdf.download(filename="01_prepared_ligand.sdf")
except FileExistsError:
    # we will raise an error if you try to overwrite an existing file; you can
    # force the file to overwrite by passing an absolute filepath instead
    pass

2024-05-07 17:26:40,721 - rush - INFO - Argument 62e4339b-546a-4335-9ca8-52df95c64768 is now ModuleInstanceStatus.RESOLVING
2024-05-07 17:26:44,258 - rush - INFO - Argument 62e4339b-546a-4335-9ca8-52df95c64768 is now ModuleInstanceStatus.ADMITTED
2024-05-07 17:26:47,782 - rush - INFO - Argument 62e4339b-546a-4335-9ca8-52df95c64768 is now ModuleInstanceStatus.DISPATCHED
2024-05-07 17:26:49,010 - rush - INFO - Argument 62e4339b-546a-4335-9ca8-52df95c64768 is now ModuleInstanceStatus.RUNNING
2024-05-07 17:27:24,581 - rush - INFO - Argument 62e4339b-546a-4335-9ca8-52df95c64768 is now ModuleInstanceStatus.AWAITING_UPLOAD


You should visualize your prepared ligand to spot check any issues

In [ ]:
view = py3Dmol.view()
with open(client.workspace / "objects" / "01_prepared_ligand.sdf", "r") as f:
    view.addModel(f.read(), "sdf")
    view.setStyle({"stick": {}})
    view.zoomTo()
    view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

# 3) Quantum energy calculation
Finally, we submit our fragmented protein for quantum energy calculation, with custom configuration.

In [ ]:
help(client.hermes_energy)

Help on function hermes_energy in module rush.provider:

hermes_energy(*args: *tuple[RushObject[Record], Record, Optional[Record], Optional[Record], Optional[Record], Optional[Record]], target: 'Target | None' = None, resources: 'Resources | None' = None, tags: 'list[str] | None' = None, restore: 'bool | None' = None) -> tuple[RushObject[Record]]
    Runs a HERMES energy calculation given a topology, and optionally model and keyword configurations.
    Will use the default model and keywords if none are provided

    Module version:
    `github:talo/tengu-prelude/3d11b61a2092fbec721966ffa99743f8bfabe6c2#hermes_energy`

    QDX Type Description:

        input: Object[Conformer];
        system: System {
            teams_per_node: u32?,
            oversubscribe_gpus: bool?,
            gpus_per_team: u32?,
            max_gpu_memory_mb: u32?
        };
        model: Model {
            basis: string,
            force_cartesian_basis_sets: bool?,
            standard_orientation: str

In [ ]:
HERMES_RESOURCES = {
    "gpus": 1,
    "storage": 100,
    "storage_units": "MB",
    "walltime": 60,
}

In [ ]:
(conformer,) = client.pick_conformer(conformer_qdxf, 0)

In [ ]:
conformer_out = json.load(open(conformer.download(overwrite=True), "r"))

2024-05-07 17:27:37,403 - rush - INFO - Argument eacda067-1323-4558-9762-54379137272e is now ModuleInstanceStatus.RESOLVING
2024-05-07 17:27:40,897 - rush - INFO - Argument eacda067-1323-4558-9762-54379137272e is now ModuleInstanceStatus.ADMITTED
2024-05-07 17:27:45,595 - rush - INFO - Argument eacda067-1323-4558-9762-54379137272e is now ModuleInstanceStatus.DISPATCHED
2024-05-07 17:27:46,748 - rush - INFO - Argument eacda067-1323-4558-9762-54379137272e is now ModuleInstanceStatus.AWAITING_UPLOAD


In [ ]:
(hermes_energy,) = client.hermes_energy(
    conformer,
    {},
    {
        "method": "RestrictedRIMP2",
        "basis": "cc-pVDZ",
        "aux_basis": "cc-pVDZ-RIFIT",
        "force_cartesian_basis_sets": True,
        "standard_orientation": "FullSystem",
    },
    None,
    None,
    None,
    resources=HERMES_RESOURCES,
)

In [ ]:
json.load(hermes_energy.download().open())

2024-05-07 17:27:59,001 - rush - INFO - Argument 1c0384f2-f487-4125-896f-17e294544dd9 is now ModuleInstanceStatus.RESOLVING
2024-05-07 17:28:01,352 - rush - INFO - Argument 1c0384f2-f487-4125-896f-17e294544dd9 is now ModuleInstanceStatus.ADMITTED
2024-05-07 17:28:03,722 - rush - INFO - Argument 1c0384f2-f487-4125-896f-17e294544dd9 is now ModuleInstanceStatus.DISPATCHED
2024-05-07 17:28:04,951 - rush - INFO - Argument 1c0384f2-f487-4125-896f-17e294544dd9 is now ModuleInstanceStatus.RUNNING
2024-05-07 17:28:17,103 - rush - INFO - Argument 1c0384f2-f487-4125-896f-17e294544dd9 is now ModuleInstanceStatus.AWAITING_UPLOAD


{'method': 'RestrictedRIMP2',
 'distance_metric': None,
 'nmers': [[{'fragments': [0],
    'density': None,
    'fock': None,
    'overlap': None,
    'h_core': None,
    'coeffs_initial': None,
    'coeffs_final': None,
    'molecular_orbital_energies': None,
    'hf_gradients': None,
    'mp2_gradients': None,
    'hf_energy': -645.0202122381007,
    'mp2_ss_correction': -0.5339090219971997,
    'mp2_os_correction': -1.4322158086920598,
    'delta_hf_energy': None,
    'delta_mp2_ss_correction': None,
    'delta_mp2_os_correction': None,
    'mulliken_charges': None,
    'fragment_distance': None,
    'bond_orders': None,
    'h_caps': None,
    'num_iters': 9,
    'num_basis_fns': 235}]],
 'reference_fragment': None,
 'expanded_hf_energy': -645.0202122381007,
 'expanded_mp2_ss_correction': -0.5339090219971997,
 'expanded_mp2_os_correction': -1.4322158086920598,
 'expanded_density': None,
 'expanded_hf_gradients': None,
 'expanded_mp2_gradients': None}